<a href="https://colab.research.google.com/github/meghamkpatel/physicaltherapyassistant/blob/main/PhysioPhrame_Test_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openai import OpenAI
from pinecone import Pinecone

# Initialize Pinecone and OpenAI services
pinecone_api_key = st.secrets["PINECONE_API_KEY"]
pc = Pinecone(api_key=pinecone_api_key)
client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

In [ ]:
# Pinecone index configuration
index_name = "physical-therapy"
index = pc.Index(index_name)

In [ ]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(index)

In [ ]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot answering questions about Physical Therapy",
)

In [ ]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

In [ ]:
testset.save("test-set.jsonl")
